# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- JG - juggling
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [2]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER

DAY_OFFS = [2,4]
DESIRED_MEAN_VALUE = 240
report_data, wide_use_data = prepare_data(PATH, DAY_OFFS, DESIRED_MEAN_VALUE)
month, mean, std, min_to_study, df = wide_use_data
df

c:\Users\San\Documents\CS projects\time_counter\preparation\calc_study_data.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Total"] = total_per_day


,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM
0,1,work,60,140,40,240,1,4,75,0,0,60
1,2,relax,0,145,15,160,2,2,100,10,10,90
2,3,work,85,145,32,262,2,0,105,0,0,0
3,4,relax,0,36,50,86,2,0,90,0,0,20
4,5,work,0,120,232,352,3,3,30,0,0,145
5,6,work,90,95,60,245,3,0,195,5,0,0
6,7,work,35,100,146,281,4,2,60,5,0,85
7,8,work,55,190,35,280,5,4,60,0,15,0
8,9,work,10,165,10,185,5,0,75,0,0,0


In [3]:
# somehow you need to figure out how to count how many days
# passed since you last had exercise of a particular intensity
# also, count how many days passed since the last time you exercised
nonstudy_df = df[["Day", "EC", "EI", "OD", "LE", "JG", "GM"]]
nonstudy_df

,Day,EC,EI,OD,LE,JG,GM
0,1,1,4,75,0,0,60
1,2,2,2,100,10,10,90
2,3,2,0,105,0,0,0
3,4,2,0,90,0,0,20
4,5,3,3,30,0,0,145
5,6,3,0,195,5,0,0
6,7,4,2,60,5,0,85
7,8,5,4,60,0,15,0
8,9,5,0,75,0,0,0


In [4]:
def get_EI_dir(df):
    """
    Summary:
        Get a dictionary that shows the most recent
        day when you had an exercise of a particular
        intensity. 
    Args:
        df of nonstudy data
    Returns:
        dir with keys representing EI level and values
        showing the most recent day of the month when
        you had EI of that level. 
    Important:
        If value equals '100', it means you didn't have
        exercise of that particular intensity
    """
    output = {}
    # move from intensity level 1 to 5
    for i in range(1, 6):
        try:
            output[i] = df[df["EI"] == i]["Day"].iloc[-1]
        except IndexError:
            print(
                f"This month doesn't have data for EI level '{i}'")
            # put 100, so later you'll get a negative number to spot
            # that you didn't have exercise of that particular level
            output[i] = 100
    return output

EI_dir = get_EI_dir(nonstudy_df)
EI_dir

This month doesn't have data for EI level '1'
This month doesn't have data for EI level '5'


{1: 100, 2: 7, 3: 5, 4: 8, 5: 100}

In [5]:
current_day = nonstudy_df["Day"].iloc[-1]
current_day

9

In [6]:
def give_EI_feedback(EI_dir, current_day):
    for key,value in EI_dir.items():
        # for every EI, count how many days passed since 
        # the last time you had exercise of that intensity
        difference = current_day - value
        print(f"EI level '{key}': last exercised", end=" ")
        # handle negative values (no data for this particular
        # month), 0 (exercised today) etc
        if difference < 0:
            print("not this month")
        elif difference == 0:
            print("today")
        elif difference == 1:
            print("1 day ago")
        elif difference > 1:
            print(f"{difference} days ago")
            
give_EI_feedback(EI_dir, current_day)

EI level '1': last exercised not this month
EI level '2': last exercised 2 days ago
EI level '3': last exercised 4 days ago
EI level '4': last exercised 1 day ago
EI level '5': last exercised not this month


In [8]:
nonstudy_df

,Day,EC,EI,OD,LE,JG,GM
0,1,1,4,75,0,0,60
1,2,2,2,100,10,10,90
2,3,2,0,105,0,0,0
3,4,2,0,90,0,0,20
4,5,3,3,30,0,0,145
5,6,3,0,195,5,0,0
6,7,4,2,60,5,0,85
7,8,5,4,60,0,15,0
8,9,5,0,75,0,0,0


In [13]:
cols = ["OD", "LE", "JG", "GM"]
total_per_col = nonstudy_df[cols].sum(
        axis=0).div(60).round(2)
total_per_col

OD    13.17
LE     0.33
JG     0.42
GM     6.67
dtype: float64

In [14]:
od, le, jg, gm = nonstudy_df[cols].sum(
        axis=0).div(60).round(2)
od, le, jg, gm

(13.17, 0.33, 0.42, 6.67)

In [7]:
# a bunch of message ideas
f"""
The last time you had exercise of intensity level {} was {}
"""
"""
The last time you had exercise of intensity level 4 was a day ago (Feb2)
"""
"The last time you juggled was 3 days ago"
"The last time you read with your left eye was 3 days ago"
"This month you played games for 30 hours"
"This month you've been outdoors for 30 hours"

SyntaxError: f-string: empty expression not allowed (3403522053.py, line 4)

In [ ]:
dir()

['DAY_OFFS',
 'DESIRED_MEAN_VALUE',
 'In',
 'Out',
 'PATH',
 '_',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'append_report',
 'calc_req_study_time',
 'calc_study_data',
 'categorise',
 'cur_stats',
 'df',
 'exit',
 'generate_report',
 'get_ipython',
 'handle_day_offs',
 'handle_nonstudy_data',
 'mean',
 'min_to_study',
 'month',
 'nonstudy_df',
 'np',
 'os',
 'pd',
 'plot_data',
 'prepare_data',
 'prepare_initial_df',
 'quit',
 'report_data',
 'std',
 'sys',
 'wide_use_data']

In [ ]:
# maybe this func is gonna belong to 'monitoring' package
def sport_stats():
    """
    (Example)
    You exercised this month: 10 times
    Last time you got medium exercise: 3 days ago
    Last time you got hard exercise: 7 days ago
    """
    pass